In [1]:
import numpy as np
import pandas as pd

In [2]:
matrix0 = pd.read_csv('./varlingam_causal_structure/adjacencymatrix0.csv', header=None)
matrix1 = pd.read_csv('./varlingam_causal_structure/adjacencymatrix1.csv', header=None)
summary_matrix = np.abs(matrix0) + np.abs(matrix1)
summary_matrix

,0,1,2,3,4,5,6,7,8,9,...,436,437,438,439,440,441,442,443,444,445
0,0.850846,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.002396,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.918205,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.997241,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.812074,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.036614,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.097351,0.000000,0.129689,0.687122,0.0,0.076591,0.0,0.000000,0.0,...,0.000000,1.226744,0.0,0.000000,0.177604,0.000000,0.000000,0.192509,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.937072,0.000000,0.000000,0.000000,0.000000
442,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.004428,0.0,...,0.000000,0.000000,0.0,0.000000,0.058385,0.032228,0.817178,0.036351,0.000000,0.000000
443,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.840504,0.000000,0.000000
444,0.000000,0.000000,0.000000,0.041749,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.046750,0.000000,0.000000,0.638197,0.139352


In [3]:
data = pd.read_csv('./data/Cleaned_S_P_500_Data.csv', delimiter=',', index_col=False, header=0)
# data.shape
print(data)

          A     AAP     AAPL    ABC    ABMD    ABT     ACN    ADBE     ADI  \
0     25.85   41.54  165.298  21.20    7.97  45.12   33.21   31.10   27.76   
1     25.22   41.50  165.180  21.01    7.94  45.00   33.33   31.00   27.56   
2     25.29   41.62  166.550  21.14    7.83  45.06   34.41   31.49   28.15   
3     25.86   41.20  170.310  21.29    7.84  45.63   34.69   32.05   28.80   
4     26.41   41.50  172.930  21.17    7.81  45.91   34.50   32.51   29.30   
...     ...     ...      ...    ...     ...    ...     ...     ...     ...   
2599  85.30  160.57  284.270  85.42  172.30  87.28  211.61  329.64  119.72   
2600  85.45  159.27  289.910  85.05  171.01  87.28  212.05  331.20  119.73   
2601  85.42  158.35  289.800  84.91  169.27  87.40  212.22  330.79  119.45   
2602  84.90  159.12  291.520  83.64  167.29  86.80  210.64  328.34  118.86   
2603  85.31  160.16  293.650  85.02  170.59  86.86  210.57  329.81  118.84   

        ADM  ...    XEC     XEL   XLNX    XOM   XRAY    XRX    

In [8]:
from sklearn.linear_model import LinearRegression

nonzero_columns = summary_matrix.apply(lambda row: row[row != 0].index.tolist(), axis=1)
predicted = {}
lags = 3
for index, columns in nonzero_columns.items():
    print(index)
    causes_df = data.iloc[:, columns]
    target_df = data.iloc[:, index]
    col_names = list(range(lags*len(columns)))
    causes_lr = pd.DataFrame(columns=col_names)
    
    for i in range(lags, causes_df.shape[0]):
        lagged_vars = causes_df.iloc[(i-lags):i]
        arrays = [lagged_vars[col].to_numpy() for col in lagged_vars]
        long_array = np.concatenate(arrays)
        causes_lr.loc[i-lags] = long_array
    target_lr = target_df.iloc[lags:]

    train_length = int(target_lr.shape[0]*0.8)
    predict_list = []
    for t in range(train_length, target_lr.shape[0]):
        model = LinearRegression()
        model.fit(causes_lr.head(t), target_lr.head(t))
        predict = model.predict([causes_lr.iloc[t]])
        predict_list.append(predict[0])
    predicted[index] = predict_list

df = pd.DataFrame(predicted)
df.to_csv('sp500_predictions_self.csv', index=False)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27